In [1]:
from compas.datastructures import Mesh
from compas.geometry import add_vectors, scale_vector, Translation
from compas.geometry import Point, Frame, Cylinder, Circle, Plane
from compas.geometry import distance_point_point

In [2]:
mesh_grid = Mesh.from_json('./cas_bcpx.json')

In [3]:
def extrude_frames(mesh, thickness=0.5):

    vertices_list = []
    key_index_a = {}
    for i, key in enumerate(mesh.vertices()):
        pt = mesh.vertex_coordinates(key)
        normal = mesh.vertex_normal(key)
        vertices_list.append(
            add_vectors(pt, scale_vector(normal, thickness * .5)))
        key_index_a[key] = i

    n = mesh.number_of_vertices()
    key_index_b = {}
    for i, key in enumerate(mesh.vertices()):
        pt = mesh.vertex_coordinates(key)
        normal = mesh.vertex_normal(key)
        vertices_list.append(
            add_vectors(pt, scale_vector(normal, thickness * -.5)))
        key_index_b[key] = i + n

    faces_list = []
    for u, v in mesh.edges():
        faces_list.append(
            [key_index_a[u], key_index_a[v], key_index_b[v], key_index_b[u]])


    return Mesh.from_vertices_and_faces(vertices_list, faces_list)

In [4]:
thickness = 0.7
mesh = extrude_frames(mesh_grid, thickness)

In [5]:
from viewer import ThreeJsViewer
viewer = ThreeJsViewer()
viewer.draw_mesh_edges(mesh, color=(1,1,0))
camera_position = [5.0, -5.0, 5.0]
viewer.show(camera_position)

/Users/kaot/anaconda3/envs/ita19/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/Users/kaot/anaconda3/envs/ita19/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

In [6]:
def cylinder_from_two_pts(p1, p2, radius=1):
    plane = Plane(p1, [p2[i] - p1[i] for i in range(3)])
    dist = distance_point_point(p1, p2)
    cylinder = Cylinder(Circle(plane, radius), dist)
    cylinder = cylinder.transformed(Translation([0.5*(p2[i] - p1[i]) for i in range(3)]))

    return cylinder

In [7]:
for vkey in mesh.vertices():
    v = mesh.vertex_coordinates(vkey)
    mesh.set_vertex_attributes(vkey, "xyz", [v[i]*20 for i in range(3)])

In [8]:
import numpy as np
import ipyvolume as ipv
from compas_vol.primitives import VolCylinder
from compas_vol.combinations import Union, SmoothUnion


vframes = []

# count = 0
for u, v in mesh.edges():
    vframe = VolCylinder(cylinder_from_two_pts(mesh.vertex_coordinates(u),
                                               mesh.vertex_coordinates(v), 4))
#     if count == 135:
#         break
#     count += 1
    print(mesh.vertex_coordinates(u))
    print(mesh.vertex_coordinates(v))
    print("---")
    vframes.append(vframe)

volframe = vframes[0]
for vf in vframes:
    volframe = SmoothUnion(volframe, vf, 3)

[-155.23398059476284, 12.677859260748075, 23.29717559247575]
[-169.81167615673542, -34.33973636389236, 22.220525742291187]
---
[-155.23398059476284, 12.677859260748075, 23.29717559247575]
[-144.09386974703403, 6.022168205072238, 18.043633275932454]
---
[-155.23398059476284, 12.677859260748075, 23.29717559247575]
[-143.67304292437433, 6.439869121726677, 50.45631209530177]
---
[-155.23398059476284, 12.677859260748075, 23.29717559247575]
[-161.12702075229873, 15.590282827380037, 2.126412192079539]
---
[-155.23398059476284, 12.677859260748075, 23.29717559247575]
[-115.99097406422405, 57.02976620677637, 26.245213778599094]
---
[86.30652722385568, -52.64774793575193, 2.0398702362964225]
[58.635431872453424, -54.40092507112665, 2.0444374729887382]
---
[86.30652722385568, -52.64774793575193, 2.0398702362964225]
[86.45928088161307, -39.25624376346682, -2.039870236296449]
---
[86.30652722385568, -52.64774793575193, 2.0398702362964225]
[86.16950416703118, -45.77124530040493, 24.396878926625657]
-

[128.50310742326639, 10.734823566170451, -2.916084638657521]
[124.96218107876399, -0.1708693249910842, 22.92489403266233]
---
[-106.56341894248652, -12.046985349009853, 19.71326611581256]
[-111.86180821254523, -9.505600023831908, 37.729044943038154]
---
[149.3903036674236, -2.033501086626993, 19.60058756678958]
[124.96218107876399, -0.1708693249910842, 22.92489403266233]
---
[-108.63481263300427, -57.207642973443996, -1.639473957932406]
[-115.13681908265112, -34.91872887768896, -1.5463920349228435]
---
[-17.895264377742247, -29.137259427246956, 46.373118580058645]
[-2.6469146434630386, -2.5684740397238315, 50.437415747840355]
---
[-95.8673264544155, -89.83240088054919, 45.323855739490014]
[-65.39525599515325, -88.02912779959023, 45.26338002034744]
---
[101.26084702887654, 7.82360041982831, 28.040302304160278]
[124.96218107876399, -0.1708693249910842, 22.92489403266233]
---
[101.26084702887654, 7.82360041982831, 28.040302304160278]
[84.29919757756849, 23.005962349342116, 28.623055438368

In [9]:
# cyl = cylinder_from_two_pts([0,0,0], [0,0,128], 128)
# print(cyl)
# volframe = VolCylinder(cyl)

In [10]:
x, y, z = np.ogrid[-300:300:128j,-300:300:128j,-300:300:128j]
dm = volframe.get_distance_numpy(x, y, z)

In [11]:
fig = ipv.figure()
meshes = ipv.plot_isosurface(dm, 0.0, color='yellowgreen', extent=[(-300,300) for _ in range(3)])
# ipv.style.use('minimal')
ipv.show()

In [12]:
from compas_vol.utilities import export_ipv_mesh

In [13]:
export_ipv_mesh(meshes, './space_frame_structure.obj')